--- Day 21: Step Counter ---
You manage to catch the airship right as it's dropping someone else off on their all-expenses-paid trip to Desert Island! It even helpfully drops you off near the gardener and his massive farm.

"You got the sand flowing again! Great work! Now we just need to wait until we have enough sand to filter the water for Snow Island and we'll have snow again in no time."

While you wait, one of the Elves that works with the gardener heard how good you are at solving problems and would like your help. He needs to get his steps in for the day, and so he'd like to know which garden plots he can reach with exactly his remaining 64 steps.

He gives you an up-to-date map (your puzzle input) of his starting position (S), garden plots (.), and rocks (#). For example:

...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
The Elf starts at the starting position (S) which also counts as a garden plot. Then, he can take one step north, south, east, or west, but only onto tiles that are garden plots. This would allow him to reach any of the tiles marked O:

...........
.....###.#.
.###.##..#.
..#.#...#..
....#O#....
.##.OS####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
Then, he takes a second step. Since at this point he could be at either tile marked O, his second step would allow him to reach any garden plot that is one step north, south, east, or west of any tile that he could have reached after the first step:

...........
.....###.#.
.###.##..#.
..#.#O..#..
....#.#....
.##O.O####.
.##.O#...#.
.......##..
.##.#.####.
.##..##.##.
...........
After two steps, he could be at any of the tiles marked O above, including the starting position (either by going north-then-south or by going west-then-east).

A single third step leads to even more possibilities:

...........
.....###.#.
.###.##..#.
..#.#.O.#..
...O#O#....
.##.OS####.
.##O.#...#.
....O..##..
.##.#.####.
.##..##.##.
...........
He will continue like this until his steps for the day have been exhausted. After a total of 6 steps, he could reach any of the garden plots marked O:

...........
.....###.#.
.###.##.O#.
.O#O#O.O#..
O.O.#.#.O..
.##O.O####.
.##.O#O..#.
.O.O.O.##..
.##.#.####.
.##O.##.##.
...........
In this example, if the Elf's goal was to get exactly 6 more steps today, he could use them to reach any of 16 garden plots.

However, the Elf actually needs to get 64 steps today, and the map he's handed you is much larger than the example map.

Starting from the garden plot marked S on your map, how many garden plots could the Elf reach in exactly 64 steps?

In [1]:
puzze_input = open("./puzzle_inputs/day21test.txt").read().split("\n")

In [3]:
# Create the adjacency matrix of the graph, the starting possition is the midle of the graph
import numpy as np
def input2chararray(squaredoc: list[str]) -> np.chararray:
    """
    Given a document with all lines with the same length, create the equivalent np.chararray object
    """

    shape = (len(squaredoc[0]), len(squaredoc))

    arr = np.chararray(shape)

    # Fill the array using the characters from the list
    for i in range(len(squaredoc)):
        for j in range(len(squaredoc[0])):
            arr[i, j] = squaredoc[i][j]

    return arr


def chararray2input(arr: np.chararray) -> list[str]:
    """
    Given a np.chararray, return a list of characters with the proper shape
    """

    # Get the shape of the chararray
    rows, cols = arr.shape

    # Initialize a list to store the characters
    squaredoc = ["" for _ in range(cols)]

    # Fill the list using characters from the chararray
    for i in range(rows):
        for j in range(cols):
            squaredoc[j] += arr[i, j].decode("utf-8")

    return squaredoc

In [12]:
map_array = input2chararray(puzze_input)
print(map_array.shape)
print(map_array)

(11, 11)
[[b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.']
 [b'.' b'.' b'.' b'.' b'.' b'#' b'#' b'#' b'.' b'#' b'.']
 [b'.' b'#' b'#' b'#' b'.' b'#' b'#' b'.' b'.' b'#' b'.']
 [b'.' b'.' b'#' b'.' b'#' b'.' b'.' b'.' b'#' b'.' b'.']
 [b'.' b'.' b'.' b'.' b'#' b'.' b'#' b'.' b'.' b'.' b'.']
 [b'.' b'#' b'#' b'.' b'.' b'S' b'#' b'#' b'#' b'#' b'.']
 [b'.' b'#' b'#' b'.' b'.' b'#' b'.' b'.' b'.' b'#' b'.']
 [b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'#' b'#' b'.' b'.']
 [b'.' b'#' b'#' b'.' b'#' b'.' b'#' b'#' b'#' b'#' b'.']
 [b'.' b'#' b'#' b'.' b'.' b'#' b'#' b'.' b'#' b'#' b'.']
 [b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.' b'.']]


In [37]:
# find the node (i,j) of the start
start = np.where(map_array == b"S")

# Create a graph for the map. Using complex numbers should be very elegant:
""" 
    -> Translate the points into grid coordinates in the complex plane, only b".", not b"#"
    -> join every vertex that is a unit of distance from other vertices

"""

# Create a list of verteces for every index in the array, if the character is not b"#"
grass_patches = np.argwhere(map_array != b"#" )

# Create the complex representation
nodes = [complex(p[0], p[1]) for p in grass_patches]

#Create the edges between adjacent vertices:
graph = {node:[] for node in nodes}

for node in nodes:
    for vertex in nodes:
        if np.linalg.norm(vertex - node) == 1:
            graph[node].append(vertex)

In [ ]:
""" 
    Now that we have the graph, we can calculate the possible end points for N steps
"""